In [ ]:
%matplotlib inline

# Object detection in the street

This file is the main notebook (temporary just a structure now of all steps that we can do) for our DL4CV project. Our aim is to detect the object in the street by means of a convolutional neural network. The detection will be like a segmentation task with find out what each pixel of the object in a photo belongs to.

The Python version:

In [ ]:
import sys
sys.version

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.nn as nn
import pandas as pd
import numpy as np
import math
import random
import os
from PIL import Image
from torch.utils.data import Dataset, sampler
from MyFolder import MyImageFolder

## Feature engineering

### This step should be to load the data images and the label images (by pixel), while doing feature engineering


--In leftImg8bit.zip, the train and test images are all 8-bit 2048*1024 pixels png type images. 

--In gtCoarse, the train and test labels are all 2048*1024 pixels png type images, which ONLY color the road in pink and the small objects on it in blue, the other pixels are black.

-- ...

In [ ]:
labelscsv = pd.read_csv("../Downloads/labels.csv")

In [ ]:
print(labelscsv.shape)
print(labelscsv)

In [ ]:
#def labeltransform(img):
#    input=transforms.ToTensor()(img)
#    target=torch.IntTensor(1024, 2048).zero_()
#    for i in range(1024):
#        for j in range(2048):
#            target[i][j]=255
#            for k in range(labelscsv.shape[0]):
#                if (labelscsv.iloc[k][2]<200 and labelscsv.iloc[k][7]/255==input[0][i][j] and labelscsv.iloc[k][8]/255==input[1][i][j] and labelscsv.iloc[k][9]/255==input[2][i][j]):
#                    target[i][j]=labelscsv.iloc[k][2]
#    return(target)

### The pre-training of the images contains the following steps:

--Using transform.Scale to rize the images into width = 256 (To be modified)

--take the center part with 224 x 224 (We know that the resnet requires an input of 224 x 224)

--transforms each pixel from [0,255] to [0,1] 
--> Are we sure we want to do this? Won't we lose resolution? 

--Normalisation for resnet

In [ ]:
transform = transforms.Compose(
#    [transforms.Scale(256),
#     [transforms.CenterCrop(224),     
     [transforms.ToTensor()])
#     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

targettransform = transforms.Compose(
#    [transforms.Scale(256),
#     [transforms.CenterCrop(224),  
     [transforms.ToTensor(),
     transforms.Normalize((0, 0, 0), (1/255, 1/255, 1/255))])
datatrain = MyImageFolder(root1='../Downloads/leftImg8bit/train', root2='../Downloads/gtFine/train' , transform = transform , target_transform= targettransform)
dataval = MyImageFolder(root1='../Downloads/leftImg8bit/val', root2='../Downloads/gtFine/val' , transform = transform , target_transform= targettransform)
datatest = MyImageFolder(root1='../Downloads/leftImg8bit/test', root2='../Downloads/gtFine/test' , transform = transform , target_transform= targettransform)


### An example of the data after feature-engineering

In [ ]:
x,y=datatrain[0]
print(x)
print(y)

## Now we construct our net

In [ ]:
from Mynet import PretrainedResNet50, MyNet
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

The hyper-parameters

In [ ]:
num_classes = labelscsv.shape[0]-1
nbtrain=len(datatrain)
nbval=len(dataval)
nbtest=len(datatest)
weightpath = "../Downloads/dlcv_weight50.pth"#testweight.pth"
batchsize = 1
useGPU = False

We take a pretrained Resnet18

In [ ]:
pretrained_net = PretrainedResNet50()
pretrained_net.load_state_dict(models.resnet50(pretrained=True).state_dict())
net = MyNet(num_classes, pretrained_net)
if useGPU:
    net.cuda()

In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)
if useGPU:
    criterion.cuda()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

trainloader = DataLoader(datatrain, batch_size=batchsize, sampler=sampler.RandomSampler(datatrain))
valloader = DataLoader(dataval, batch_size=batchsize)
testloader = DataLoader(datatest, batch_size=batchsize)

We take a pretrained model if it exists.

In [ ]:
import os.path
if os.path.exists(weightpath):
    net.load_state_dict(torch.load(weightpath))

Run the net

In [ ]:
print("Start training")

for epoch in range(1):  # loop over the dataset multiple times
       
    #--------------------------------------training period---------------------------------------
    running_loss = 0.0
    epochloss = 0.0
    numsample=0
    printfrequence=10
    
    net.train()
    for inputs, labels in trainloader:
        if useGPU:
            inputs = inputs.cuda()
            labels = labels.cuda()
        inputs, labels = Variable(inputs), Variable(labels)
   
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        epochloss+=loss.data[0]
        numsample += batchsize
        if numsample % printfrequence == 0: #printfrequence-1: 
            print('[%d, %5d] loss: %.5f' % (epoch+1, numsample, running_loss / printfrequence))
            running_loss = 0.0
    print('The average loss of epoch ', epoch+1, ' is ', epochloss/nbtrain)
    torch.save(net.state_dict(),weightpath)

In [ ]:
print("Start training")
newseuil=torch.Tensor(numlabels)

ConfusionMatrixTrain=[torch.Tensor() for i in range(numlabels)]
ConfusionMatrixValidation=[torch.Tensor() for i in range(numlabels)]
ConfusionMatrixTest=[torch.Tensor() for i in range(numlabels)]
APM=Meter.APMeter()

for epoch in range(2):  # loop over the dataset multiple times
       
    #--------------------------------------training period---------------------------------------
    running_loss = 0.0
    epochloss = 0.0
    numsample=0
    printfrequence=10000
    for i in range(numlabels):
        ConfusionMatrixTrain[i]=torch.IntTensor(2,2).zero_()
    
    net.train()
    for inputs, labels in trainloader:
        if useGPU:
            inputs = inputs.cuda()
            labels = labels.cuda()
        inputs, labels = Variable(inputs), Variable(labels)
   
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #trainging confusion matrix
        result=FinalResult(outputs.data.cpu(), seuil.repeat(outputs.data.cpu().size(0),1))
        for j in range(labels.data.cpu().size(0)):
            for i in range(numlabels):
                ConfusionMatrixTrain[i][int(labels.data.cpu()[j][i])][int(result.cpu()[j][i])]+=1

        # print statistics
        running_loss += loss.data[0]
        epochloss+=loss.data[0]
        numsample += batchsize
        if numsample % printfrequence == 0: #printfrequence-1:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.5f' % (epoch+1, numsample, running_loss / printfrequence))
            running_loss = 0.0
    print('The average loss of epoch ', epoch+1, ' is ', epochloss/nbtrain)
    print('The confusion matrixs for training are: ', ConfusionMatrixTrain)
    traingraph.append(epochloss/nbtrain)
    ConMatTrain.append(ConfusionMatrixTrain.copy())
    torch.save(net.state_dict(),weightpath)
    
    #--------------------------------------Validation period---------------------------------------
    
    running_loss = 0.0
    epochloss = 0.0
    numsample=0
    printfrequence=10000
    for i in range(numlabels):
        newseuil[i]=0.0
        ConfusionMatrixValidation[i]=torch.IntTensor(2,2).zero_()
    
    net.eval()
    for inputs, labels in validationloader:
        if useGPU:
            inputs = inputs.cuda()
        outputs=net(Variable(inputs))
        result=FinalResult(outputs.data.cpu(), seuil.repeat(outputs.data.cpu().size(0),1))
        
        #validation confusion matrix
        for j in range(labels.cpu().size(0)):
            newseuil+=outputs.data.cpu()[j]
            numsample+=1
            for i in range(numlabels):
                ConfusionMatrixValidation[i][int(labels.cpu()[j][i])][int(result.cpu()[j][i])]+=1

  
    newseuil=newseuil/numsample
    seuil=newseuil.clone()
    print('The seuil of epoch', epoch+1, ' is defined at', seuil)
    ConMatValidation.append(ConfusionMatrixValidation.copy())
    #print('The confusion matrixs for validation are: ', ConfusionMatrixvalidation)
    
    #--------------------------------------Test period---------------------------------------
    
    APM.reset()
    numsample=0
    printfrequence=10000
    classerror=torch.zeros(numlabels)
    for i in range(numlabels):
        ConfusionMatrixTest[i]=torch.IntTensor(2,2).zero_()
        
    net.eval()
    for inputs, labels in testloader:
        if useGPU:
            inputs = inputs.cuda()
            
        outputs=net(Variable(inputs))
        result=FinalResult(outputs.data.cpu(), seuil.repeat(outputs.data.cpu().size(0),1))
        
        for j in range(labels.size(0)):
            classerror.add_(torch.Tensor.float((labels[j]-result.cpu()[j]).abs()))
            numsample +=1
            APM.add(outputs.data.cpu()[j], labels[j])
            for i in range(numlabels):
                ConfusionMatrixTest[i][int(labels[j][i])][int(result.cpu()[j][i])]+=1
        
        if numsample % printfrequence == 0: #printfrequence-1:    # print every 100 mini-batches
            print(numsample, 'images passed')

    
    print('In total we have ', numsample, 'images for the test.')
    print("The Average Precision are ", APM.value())
    #print("The correctness per class are ", torch.ones(numlabels).sub(classerror.div_(numsample)))
    print("The mean test loss is ", classerror.div_(numsample).mean())
    print('The confusion matrixs for test are: ', ConfusionMatrixTest)
    ConMatTest.append(ConfusionMatrixTest.copy())
    testgraph.append(classerror.mean())

In [ ]:
import matplotlib.pyplot as plt
#plt.plot([1,2,3,4])
plt.axis([0, 10, 0, 0.5])

plt.plot(traingraph, label="training loss")
plt.plot(testgraph, label="test loss")
legend = plt.legend()

plt.ylabel('mean loss')
plt.show()

Analysis the result

In [ ]:
import matplotlib.pyplot as plt
#plt.plot([1,2,3,4])
plt.axis([0, 10, 0.75, 1])

x1=[0.75, 0.9859, 0.9938, 0.9926, 0.9940, 0.9943, 0.9946, 0.9953, 0.9987]
x2=[0.75, 0.8497, 0.8810, 0.9054, 0.8920, 0.9360, 0.9500, 0.9181, 0.9349]
x3=[0.75, 0.9711, 0.9800, 0.9848, 0.9856, 0.9930, 0.9890, 0.9922, 0.9943]
x4=[0.75, 0.9674, 0.9591, 0.9761, 0.9897, 0.9834, 0.9881, 0.9974, 0.9894]

y1=[0.9791, 0.9748, 0.9785, 0.9635, 0.9793, 0.9803, 0.9625, 0.9768, 0.9773]
y2=[0.7766, 0.8092, 0.8271, 0.8145, 0.8455, 0.8559, 0.8602, 0.8586, 0.8637]
y3=[0.9328, 0.9251, 0.9265, 0.9537, 0.9620, 0.9511, 0.9432, 0.9288, 0.9583]
y4=[0.9470, 0.9358, 0.9466, 0.9527, 0.9540, 0.9533, 0.9613, 0.9481, 0.9616]

plt.plot(x1, label="train lisse-dente")
plt.plot(x2, label="train alterne-oppose")
plt.plot(x3, label="train simple-composee")
plt.plot(x4, label="train non ligneux-ligneux")

plt.plot(y1, label="test lisse-dente")
plt.plot(y2, label="test alterne-oppose")
plt.plot(y3, label="test simple-composee")
plt.plot(y4, label="test non ligneux-ligneux")

legend = plt.legend()

plt.xlabel('epoch')
plt.ylabel('Correctness par class')
plt.show()